# Import Libraries

In [1]:
import sys
import os

notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [2]:
from text_to_sql import (
    TextToSQL,
    Config,
    LLMConfig,
    SLConfig,
    ContextConfig,
    QueryConfig,
)
from dotenv import load_dotenv
from datetime import datetime

import pandas as pd
import os

# Constants

In [3]:
MAX_RETRIES = 5
RETRY_DELAY = 2
DATABASE = "northwind"
MODEL = "deepseek-chat"
PROVIDER = "deepseek"

# Load Environment

In [4]:
load_dotenv()

True

# Set Timestamp Experiment

In [5]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
output_dir = f"../files/experiment_result/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

# Config

In [6]:
db_key = DATABASE.upper().replace("-", "_")
provider_key = PROVIDER.upper().replace("-", "_")

config = Config(
    max_retry_attempt=5,
    rewriter_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    query_generator_config=LLMConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
    ),
    schema_linker_config=SLConfig(
        type="api",
        model=MODEL,
        provider=PROVIDER,
        api_key=os.getenv(f"API_KEY_{provider_key}"),
        schema_path=f"../files/schema/{DATABASE}.txt",
        metadata_path=f"../files/metadata/{DATABASE}.json",
    ),
    retrieve_context_config=ContextConfig(data_path=f"../files/dataset/dataset_{DATABASE}.csv"),
    query_executor_config = QueryConfig(
        host=os.getenv(f"DB_HOST_{db_key}"),
        database=os.getenv(f"DB_DATABASE_{db_key}"),
        user=os.getenv(f"DB_USER_{db_key}"),
        password=os.getenv(f"DB_PASSWORD_{db_key}"),
        port=os.getenv(f"DB_PORT_{db_key}"),
    ),
)

# Model

In [7]:
text_to_sql_model = TextToSQL(config=config)

Initializing API client for deepseek using model deepseek-chat.
Initializing API client for deepseek using model deepseek-chat.
Initializing API client for deepseek using model deepseek-chat.


c:\Users\michj\AppData\Local\Programs\Python\Python39\lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


# Import Dataset

In [8]:
dataset = pd.read_csv(f"../files/dataset/dataset_{DATABASE}.csv")

In [9]:
dataset

,Question,Answer,Summary,Alternative Prompt 1 (English),Alternative Prompt 2 (Bahasa Indonesia),Expected Result
0,How much total revenue did we make in 1997? Sh...,SELECT SUM(od.unit_price * od.quantity * (1.0 ...,This SQL query calculates the total revenue ge...,How much money did we make in 1997 total? Show...,Total pendapatan kita tahun 1997 berapa sih? T...,['total_revenue_1997']
1,How much has each customer paid us in total? S...,"SELECT c.company_name, SUM(od.unit_price * od....",This SQL query calculates the total revenue ge...,How much has each customer paid overall? Show ...,Tiap customer udah bayar ke kita totalnya bera...,"['company_name', 'total']"
2,What are the top 10 best-selling products base...,"SELECT p.product_name, SUM(od.unit_price * od....",This SQL query identifies the top 10 best-sell...,"What are our top 10 most sold products, lookin...",10 produk paling laku berdasarkan pendapatan s...,"['product_name', 'sales']"
3,Which customers from the UK have paid us more ...,"SELECT c.contact_name,\n SUM(od.unit_pri...",This SQL query identifies UK-based customers w...,Which UK customers paid more than $1000? Show ...,Customer dari UK yang bayar lebih dari $1000 s...,"['contact_name', 'payments']"
4,How much revenue has each customer generated i...,"SELECT c.customer_id, c.company_name, c.countr...",This SQL query analyzes customer spending patt...,"How much has each customer paid in total, incl...",Berapa total pembayaran yang dilakukan oleh se...,"['customer_id', 'company_name', 'country', 'cu..."
5,Which top 1 employee have handled the most ord...,"SELECT e.employee_id, e.first_name, e.last_nam...",This SQL query identifies the top 1 employee b...,Which top 1 employee handled the most orders? ...,Top 1 Karyawan mana yang paling banyak urus or...,"['employee_id', 'first_name', 'last_name', 'to..."
6,What are the top 5 countries by total number o...,"SELECT country, COUNT(customer_id) AS total_cu...",This SQL query identifies the top 5 countries ...,Top 5 countries with the most customers? Show ...,5 negara dengan jumlah customer terbanyak? Tam...,"['country', 'total_customers']"
7,Which top 1 supplier provide the most products...,"SELECT s.supplier_id, s.company_name, COUNT(p....",This SQL query analyzes supplier product offer...,Which top 1 supplier offer the most products? ...,Top 1 supplier mana yang punya produk paling b...,"['supplier_id', 'company_name', 'product_count']"
8,What are the total units sold per product? Sho...,"SELECT p.product_id, p.product_name, SUM(od.qu...",This SQL query analyzes product sales performa...,How many units sold for each product? Show 'pr...,Total unit terjual per produk? Tampilkan 'prod...,"['product_id', 'product_name', 'total_units_so..."
9,Which top 1 region has the most territories? S...,"SELECT r.region_id, COUNT(t.territory_id) AS t...",This SQL query counts the number of territorie...,Which top 1 region have the most territories? ...,Top 1 region mana yang punya territory paling ...,"['region_id', 'total_territories']"


# Experiment V3

In [10]:
import time
import ast

EA = 0
total_questions = len(dataset) * 2
results_list = []

for idx, row in dataset.iterrows():
    question_1 = row["Alternative Prompt 1 (English)"]
    question_2 = row["Alternative Prompt 2 (Bahasa Indonesia)"]
    answer = row["Answer"]
    expected_columns = ast.literal_eval(row["Expected Result"])

    for prompt_id, question in enumerate([question_1, question_2], start=1):
        print(f"\nProcessing Question {idx + 1}.{prompt_id}: {question}")
        result = None

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                result = text_to_sql_model.generate_v3(user_prompt=question)
                break
            except Exception as e:
                print(f"[Attempt {attempt}] Failed to generate SQL: {e}")
                if attempt < MAX_RETRIES:
                    time.sleep(RETRY_DELAY)
                else:
                    print("Max retries reached. Setting result as 'ERROR'")
                    result = "ERROR"

        print(f"Generated SQL Query: {result}")

        try:
            acc = text_to_sql_model.evaluate(query=result, true_query=answer, expected_columns=expected_columns)
        except Exception as e:
            print(f"Evaluation failed: {e}")
            acc = 0.0

        print(f"Execution Accuracy: {acc:.4f}")

        results_list.append({
            "Question ID": f"{idx + 1}.{prompt_id}",
            "Question": question,
            "Generated SQL Query": result,
            "Expected SQL Query": answer,
            "Execution Accuracy": acc
        })

        EA += acc

# Calculate final execution accuracy
final_accuracy = EA / total_questions if total_questions > 0 else 0
print(f"\nFinal Execution Accuracy: {final_accuracy:.4f}")


Processing Question 1.1: How much money did we make in 1997 total? Show it as '1997 Total Revenues'.
Generated SQL Query: SELECT SUM(od.unit_price * od.quantity * (1.0 - od.discount)) AS "1997 Total Revenues" 
FROM order_details od 
INNER JOIN orders o ON od.order_id = o.order_id 
WHERE EXTRACT(YEAR FROM o.order_date) = 1997;
Execution Accuracy: 1.0000

Processing Question 1.2: Total pendapatan kita tahun 1997 berapa sih? Tampilkan sebagai '1997 Total Revenues'.
Generated SQL Query: SELECT SUM(od.unit_price * od.quantity * (1.0 - od.discount)) AS "1997 Total Revenues" 
FROM order_details od 
INNER JOIN orders o ON od.order_id = o.order_id 
WHERE EXTRACT(YEAR FROM o.order_date) = 1997;
Execution Accuracy: 1.0000

Processing Question 2.1: How much has each customer paid overall? Show 'CompanyName' and total as 'Total'.
Generated SQL Query: SELECT c.company_name AS "CompanyName", SUM(od.unit_price * od.quantity * (1.0 - od.discount)) AS "Total" 
FROM customers c 
INNER JOIN orders o ON c

In [11]:
df_results_v3 = pd.DataFrame(results_list)
df_results_v3.to_csv(f"{output_dir}/{MODEL}_{DATABASE}_v3.csv", index=False)